<a href="https://colab.research.google.com/github/Zelechos/Pragmatic_IA/blob/master/Pytorch/Datasets_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datasets Pytorch

In [3]:
import torch

In [4]:
from sklearn.datasets import fetch_openml
import numpy as np

# descarga datos

mnist = fetch_openml('mnist_784', version=1)

X, Y = mnist["data"], mnist["target"]

X_train, X_test, y_train, y_test = X[:60000] / 255., X[60000:] / 255., Y[:60000].astype(np.int), Y[60000:].astype(np.int)

#creamos los tensores 
X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.from_numpy(y_train).long().cuda()

In [5]:
from sklearn.metrics import accuracy_score

# es una funcion generalizadora
def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

def evaluate(x):
    model.eval()
    y_pred = model(x)
    y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

In [6]:
Capa_Entrada, Capa_Oculta, Capa_Salida = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(Capa_Entrada, Capa_Oculta),
    torch.nn.ReLU(),
    torch.nn.Linear(Capa_Oculta, Capa_Salida),
).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1): 
    
    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())
    
    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()
    
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.76640
Epoch 20/100 Loss 1.48378
Epoch 30/100 Loss 1.24717
Epoch 40/100 Loss 1.05596
Epoch 50/100 Loss 0.92119
Epoch 60/100 Loss 0.83264
Epoch 70/100 Loss 0.75885
Epoch 80/100 Loss 0.70330
Epoch 90/100 Loss 0.65592
Epoch 100/100 Loss 0.61634


0.9289

## Utilizaremos el Algorimto de mini-batch gradient decent

In [7]:
Capa_Entrada, Capa_Oculta, Capa_Salida = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(Capa_Entrada, Capa_Oculta),
    torch.nn.ReLU(),
    torch.nn.Linear(Capa_Oculta, Capa_Salida),
).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 10
batch_size = 100
log_each = 1
l = []
model.train()

batches = len(X_t) // batch_size

for e in range(1, epochs+1): 
    
    _l = []
    # iteramos por batches
    for b in range(batches):
        x_b = X_t[b*batch_size:(b+1)*batch_size]
        y_b = Y_t[b*batch_size:(b+1)*batch_size]
        
        # forward
        y_pred = model(x_b)

        # loss
        loss = criterion(y_pred, y_b)
        _l.append(loss.item())

        # ponemos a cero los gradientes
        optimizer.zero_grad()

        # Backprop (calculamos todos los gradientes automáticamente)
        loss.backward()

        # update de los pesos
        optimizer.step()
    
    l.append(np.mean(_l))
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 1/10 Loss 0.29818
Epoch 2/10 Loss 0.20834
Epoch 3/10 Loss 0.16724
Epoch 4/10 Loss 0.14176
Epoch 5/10 Loss 0.12383
Epoch 6/10 Loss 0.11018
Epoch 7/10 Loss 0.09928
Epoch 8/10 Loss 0.09028
Epoch 9/10 Loss 0.08279
Epoch 10/10 Loss 0.07641


0.9735

# La clase Dataset
### La primera clase que tenemos que conocer es la clase Dataset. Esta clase hereda de la clase madre torch.utils.data.Dataset y tenemos que definir, como mínimo, tres funciones:

- __init__: el constructor
- __len__: devuelve el número de muestras en el dataset
- __getitem__: devuelve una muestra en concreto del dataset
### Una vez definida la clase, ésta puede usarse como si de cualquier iterador se tratase.

In [8]:
# clase Dataset, hereda de la clase `torch.utils.data.Dataset`
class Dataset(torch.utils.data.Dataset):
    
    # constructor
    def __init__(self, X, Y): #esta clase Dataset recive como parametros arrays numpy RECORDAR
        self.X = torch.from_numpy(X).float().cuda()
        self.Y = torch.from_numpy(Y).long().cuda()
    
    # devolvemos el número de datos en el dataset
    def __len__(self):
        return len(self.X)
    
    # devolvemos el elemento `ix` del dataset
    def __getitem__(self, ix):
        return self.X[ix], self.Y[ix]

In [23]:
print(type(X_train))
print(type(y_train))

dataset = Dataset(X_train,y_train)
print(type(dataset))
print("Este es un tensor de Pytroch recuerdalo ->  ",dataset)
print("Longitud del Dataset : ",len(dataset))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class '__main__.Dataset'>
Este es un tensor de Pytroch recuerdalo ->   <__main__.Dataset object at 0x7f1a248c9c50>
Longitud del Dataset :  60000


In [16]:
Capa_Entrada, Capa_Oculta, Capa_Salida = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(Capa_Entrada, Capa_Oculta),
    torch.nn.ReLU(),
    torch.nn.Linear(Capa_Oculta, Capa_Salida),
).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 10
batch_size = 100
log_each = 1
l = []
model.train()

batches = len(dataset) // batch_size

for e in range(1, epochs+1): 
    
    _l = []
    # iteramos por batches en el dataset
    for b in range(batches):
        x_b, y_b = dataset[b*batch_size:(b+1)*batch_size]
        
        # forward
        y_pred = model(x_b)

        # loss
        loss = criterion(y_pred, y_b)
        _l.append(loss.item())

        # ponemos a cero los gradientes
        optimizer.zero_grad()

        # Backprop (calculamos todos los gradientes automáticamente)
        loss.backward()

        # update de los pesos
        optimizer.step()
    
    l.append(np.mean(_l))
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 1/10 Loss 0.30245
Epoch 2/10 Loss 0.20993
Epoch 3/10 Loss 0.16794
Epoch 4/10 Loss 0.14256
Epoch 5/10 Loss 0.12466
Epoch 6/10 Loss 0.11090
Epoch 7/10 Loss 0.09993
Epoch 8/10 Loss 0.09087
Epoch 9/10 Loss 0.08323
Epoch 10/10 Loss 0.07664


0.9761

# Clase Dataloader

In [18]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)

In [24]:
# con esto lo iteramos a nuestro data loader y le pedimos el primer batch
x, y = next(iter(dataloader))
print(type(dataloader))
print(len(dataloader))
print(x.shape, y.shape)

<class 'torch.utils.data.dataloader.DataLoader'>
600
torch.Size([100, 784]) torch.Size([100])


In [25]:
Capa_Entrada, Capa_Oculta, Capa_Salida = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(Capa_Entrada, Capa_Oculta),
    torch.nn.ReLU(),
    torch.nn.Linear(Capa_Oculta, Capa_Salida),
).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 10
log_each = 1
l = []
model.train()
for e in range(1, epochs+1): 
    
    _l = []
    # iteramos por batches en el dataloader
    for x_b, y_b in dataloader:
        
        # forward
        y_pred = model(x_b)

        # loss
        loss = criterion(y_pred, y_b)
        _l.append(loss.item())

        # ponemos a cero los gradientes
        optimizer.zero_grad()

        # Backprop (calculamos todos los gradientes automáticamente)
        loss.backward()

        # update de los pesos
        optimizer.step()
    
    l.append(np.mean(_l))
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 1/10 Loss 0.30650
Epoch 2/10 Loss 0.21272
Epoch 3/10 Loss 0.17054
Epoch 4/10 Loss 0.14479
Epoch 5/10 Loss 0.12681
Epoch 6/10 Loss 0.11327
Epoch 7/10 Loss 0.10250
Epoch 8/10 Loss 0.09355
Epoch 9/10 Loss 0.08614
Epoch 10/10 Loss 0.07971


0.9738

In [27]:
#También permite definir nuestra propia lógica para crear los batches, algo que puede ser útil en ciertas ocasiones.
def collate_fn(batch): #crear nuestros propios batches
    return torch.stack([x for x, y in batch]), torch.stack([y for x, y in batch]), torch.stack([2.*x for x, y in batch])

In [28]:
#implementar al dataloader nuestros propios batches creados
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True, collate_fn=collate_fn)

In [29]:
Capa_Entrada, Capa_Oculta, Capa_Salida = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(Capa_Entrada, Capa_Oculta),
    torch.nn.ReLU(),
    torch.nn.Linear(Capa_Oculta, Capa_Salida),
).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 10
log_each = 1
l = []
model.train()
for e in range(1, epochs+1): 
    
    _l = []
    # iteramos por batches en el dataloader
    # no usamos x2_b, sólo es para ver un ejemplo
    for x_b, y_b, x2_b in dataloader:
        
        # forward
        y_pred = model(x_b)

        # loss
        loss = criterion(y_pred, y_b)
        _l.append(loss.item())

        # ponemos a cero los gradientes
        optimizer.zero_grad()

        # Backprop (calculamos todos los gradientes automáticamente)
        loss.backward()

        # update de los pesos
        optimizer.step()
    
    l.append(np.mean(_l))
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 1/10 Loss 0.30384
Epoch 2/10 Loss 0.21194
Epoch 3/10 Loss 0.17112
Epoch 4/10 Loss 0.14620
Epoch 5/10 Loss 0.12906
Epoch 6/10 Loss 0.11575
Epoch 7/10 Loss 0.10521
Epoch 8/10 Loss 0.09640
Epoch 9/10 Loss 0.08923
Epoch 10/10 Loss 0.08308


0.9754